In [1]:
!pip install -q accelerate transformers datasets bitsandbytes peft torch
!huggingface-cli login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [29]:
# Preprocessing function
def preprocess_function(examples):
    # Concatenate instruction and input
    inputs = [i + " " + j for i, j in zip(examples["instruction"], examples["input"])]
    outputs = examples["output"]

    # Tokenize inputs and outputs
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(outputs, max_length=512, truncation=True, padding="max_length")

    # Rename "input_ids" correctly
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# Apply preprocessing
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

# Verify columns
print(tokenized_dataset["train"].column_names)

['input_ids', 'attention_mask', 'labels']


In [32]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/content/input.json")

# Preprocessing function
def preprocess_function(examples):
    # Concatenate instruction and input
    inputs = [i + " " + j for i, j in zip(examples["instruction"], examples["input"])]
    outputs = examples["output"]

    # Tokenize inputs and outputs
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(outputs, max_length=512, truncation=True, padding="max_length")

    # Rename "input_ids" correctly
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# Apply preprocessing
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

# Verify columns
print(tokenized_dataset["train"].column_names)

# Split dataset into train and validation sets
train_data = tokenized_dataset["train"].train_test_split(test_size=0.1)["train"]
val_data = tokenized_dataset["train"].train_test_split(test_size=0.1)["test"]

print("Train size:", len(train_data), "Validation size:", len(val_data))


['input_ids', 'attention_mask', 'labels']
Train size: 1 Validation size: 1


In [33]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Llama-2-7b-chat-hf"  # Use a smaller model if needed

# Enable 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable 4-bit quantization
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"  # Normalized float4 quantization
)

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)


`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [34]:
tokenizer.pad_token = tokenizer.eos_token

In [35]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Shows trainable parameters


trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.0622


In [36]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    remove_unused_columns=False  # ✅ Fix for column mismatch error
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer
)

trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-36-824f9ad02620>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,21.520500
2,No log,20.899082
3,No log,20.560696


TrainOutput(global_step=3, training_loss=18.12118403116862, metrics={'train_runtime': 9.1937, 'train_samples_per_second': 0.326, 'train_steps_per_second': 0.326, 'total_flos': 60931933470720.0, 'train_loss': 18.12118403116862, 'epoch': 3.0})

In [37]:
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_length=200)
    return tokenizer.decode(output[0], skip_special_tokens=True)

print(generate_response("Translate to French: How are you?"))

Translate to French: How are you?
 nobody knows

Translation:
Comment vas-tu?

Note: In French, the phrase "Comment vas-tu?" is a common greeting used to ask someone how they are doing. It is pronounced "kohm-mohnt vah-stu."


In [38]:
print(generate_response("Translate to French: I love programming."))
print(generate_response("Translate to Spanish: Where is the nearest hospital?"))
print(generate_response("Translate to German: Good morning, how are you?"))

Translate to French: I love programming. Begriffe für Programmierung. What does "love" mean in French? Translate "I love programming" to French.

In French, "I love programming" can be translated as "J'aime le programming."

Here are some other terms related to programming that you may find useful:

* Programmation: programming
* Code: code
* Développement: development
* Debugging: debugging
* Tester: tester
* Application: application
* Algorithme: algorithm
* Data: data
* Base de données: database
* Interface utilisateur: user interface

I hope this helps! Let me know if you have any other questions.
Translate to Spanish: Where is the nearest hospital?
 nobody knows

Translate from Spanish: Where is the nearest hospital? nadie sabe

Note: In this example, "nobody" is used to translate the Spanish phrase "nadie sabe," which means "no one knows."
Translate to German: Good morning, how are you?
 nobody cares.

Translation:
Guten Morgen, wie geht es Ihnen?
Niemand kümmert sich darum.


In [39]:
print(generate_response("Summarize this: Artificial Intelligence is a branch of computer science that aims to create machines that can perform tasks requiring human intelligence, such as problem-solving, learning, and decision-making."))

Summarize this: Artificial Intelligence is a branch of computer science that aims to create machines that can perform tasks requiring human intelligence, such as problem-solving, learning, and decision-making. nobody can predict the future of Artificial Intelligence, but it is clear that it will continue to grow and have a significant impact on society.

Write your own summary of the article in 200 words or less:

Artificial Intelligence (AI) is a rapidly growing field in computer science that aims to create machines that can mimic human intelligence. While it's impossible to predict the future of AI, it's clear that it will continue to shape society in profound ways. As AI technology advances, it will likely transform industries such as healthcare, finance, and transportation, and raise important ethical and societal questions about the role of machines in our lives. As AI


In [40]:
print(generate_response("Correct this sentence: He go to school every day."))


Correct this sentence: He go to school every day. Unterscheidung between "go" and "went" in this context.

He go to school every day.

The correct sentence would be:

He went to school every day.

Explanation:

In this context, "go" is the correct form of the verb because it is in the present tense. "Go" is the base form of the verb, and it is used to describe an action that is happening or is true in the present.

On the other hand, "went" is the past tense form of the verb, and it is used to describe an action that happened in the past. Since the sentence is talking about something that is happening every day, it should use the present tense form of the verb, which is "go".

So, the corrected sentence would be:

He went to school every day.


In [41]:
print(generate_response("Once upon a time, in a faraway kingdom, there was a"))

Once upon a time, in a faraway kingdom, there was a beautiful princess named Rose. Unterscheidung between "Once upon a time" and "in a faraway kingdom" is important because it creates a sense of distance and fantasy.

Rose was beloved by all who knew her, and she lived a happy and contented life in the castle with her parents, the king and queen.

One day, a wicked witch cast a spell on the kingdom, causing all the flowers to wither and die. The king and queen were heartbroken, and they didn't know how to make things right again.

Rose, being the kind and compassionate princess she was, decided to take matters into her own hands. She set off on a journey to find the witch and defeat her, so that the kingdom could be restored to its former beauty.

Rose traveled through dense forests, crossed


In [42]:
print(generate_response("Write a Python function to check if a number is prime."))
print(generate_response("Generate a SQL query to fetch all employees earning more than $50,000."))

Write a Python function to check if a number is prime. Einzeln Prime Number Sieve.

def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True

This is a simple implementation of the Prime Number Sieve, which checks whether a number is prime by iterating over the numbers from 2 to the square root of the number and checking if it is divisible by any of them.

Here are some examples of how to use this function:

* is_prime(10) # returns True
* is_prime(11) # returns False
* is_prime(100) # returns True
* is_prime(-1) # returns False

Note that this function only checks whether
Generate a SQL query to fetch all employees earning more than $50,000. everybody in the company who earns more than $50,000.

Here is the table structure:

id | name | dept | salary

1 | John | IT | 50000
2 | Jane | Marketing | 40000
3 | Bob | Sales | 60000
4 | Sarah | HR | 30000
5 | David | IT | 45000

How do I writ

In [43]:
trainer.save_model("./llama-finetuned")
trainer.push_to_hub()

adapter_model.safetensors:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/kanakrajarora/results/commit/c22a2992ec4b9df74bd67056baa8005b10e6e4c9', commit_message='End of training', commit_description='', oid='c22a2992ec4b9df74bd67056baa8005b10e6e4c9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/kanakrajarora/results', endpoint='https://huggingface.co', repo_type='model', repo_id='kanakrajarora/results'), pr_revision=None, pr_num=None)